In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.colab import files

uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]

Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960.json


In [ ]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")

Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1WkuJI5YKIBFA4DNB62_K4bsi_oku2EYsH-Iaqqy1cCc


In [ ]:
# Step 5: Define Meeting Notes
meeting_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
# Step 6: Function to Add Content to Google Doc
def add_text_to_doc(doc_id, text, style='NORMAL_TEXT'):
    requests = [
        {
            'insertText': {
                'location': {'index': 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': 1,
                    'endIndex': len(text) + 2
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()


In [ ]:
# Step 7: Parse Markdown and Format Content
import re

def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('# '):  # Heading 1
            add_text_to_doc(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_to_doc(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_to_doc(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_text_to_doc(doc_id, f"☐ {line[6:]}", style='NORMAL_TEXT')
        elif line.startswith('-'):  # Bullet point
            add_text_to_doc(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_to_doc(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_to_doc(doc_id, line, style='NORMAL_TEXT')

In [ ]:
# Step 8: Add Content to the Document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/1WkuJI5YKIBFA4DNB62_K4bsi_oku2EYsH-Iaqqy1cCc


In [ ]:
# Step 9: Add Error Handling
try:
    # Final content check and print the document link
    print("Document successfully created and formatted!")
except Exception as e:
    print(f"An error occurred: {e}")

Document successfully created and formatted!


In [ ]:
# Share the document with your email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

def share_document_with_user(doc_id, email):
    # Set file permissions
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)


Document shared with dani.s.ankit@gmail.com


In [ ]:
# v2

In [ ]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]

Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960 (1).json


In [ ]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")


Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/15dAEacXn-GkZPSC9SOl4zyA2xtQq8C7Q06diL5ga-bI


In [ ]:

# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)


Document shared with dani.s.ankit@gmail.com


In [ ]:
# Step 6: Define Meeting Notes
meeting_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
# Step 7: Add Content to Google Doc
def add_text_to_doc(doc_id, text, style='NORMAL_TEXT'):
    requests = [
        {
            'insertText': {
                'location': {'index': 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': 1,
                    'endIndex': len(text) + 2
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Parse and add content with appropriate formatting
def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('# '):  # Heading 1
            add_text_to_doc(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_to_doc(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_to_doc(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_text_to_doc(doc_id, f"☐ {line[6:]}", style='NORMAL_TEXT')
        elif line.startswith('-'):  # Bullet point
            add_text_to_doc(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_to_doc(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_to_doc(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/15dAEacXn-GkZPSC9SOl4zyA2xtQq8C7Q06diL5ga-bI


In [ ]:
# v3 - correct order

In [ ]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:

# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]  # Get the uploaded file name


Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960 (2).json


In [ ]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")

Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")

# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)

Document created! View it here: https://docs.google.com/document/d/1vdm36ZayGHquEQ9cVuOxKO9-UdjNTCADCnG3wO6jF-Q
Document shared with dani.s.ankit@gmail.com


In [ ]:
# Step 6: Define Meeting Notes
meeting_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
# Step 7: Add Content to Google Doc
def add_text_to_doc(doc_id, text, style='NORMAL_TEXT'):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to insert text at the end
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text) - 1
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Parse and add content with appropriate formatting
def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('# '):  # Heading 1
            add_text_to_doc(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_to_doc(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_to_doc(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_text_to_doc(doc_id, f"☐ {line[6:]}", style='NORMAL_TEXT')
        elif line.startswith('-'):  # Bullet point
            add_text_to_doc(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_to_doc(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_to_doc(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/1vdm36ZayGHquEQ9cVuOxKO9-UdjNTCADCnG3wO6jF-Q


In [ ]:
# v4 - correct checkbox

In [ ]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]  # Get the uploaded file name


Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960 (3).json


In [ ]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")


Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1SckjOLZIpmhA166h4eNEUoUuB4KPvatls4wyvZ6t2Jw


In [ ]:
# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)

Document shared with dani.s.ankit@gmail.com


In [ ]:
# Step 6: Define Meeting Notes
meeting_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
# Step 7: Add Content to Google Doc
def add_text_to_doc(doc_id, text, style='NORMAL_TEXT'):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to insert text at the end
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text) - 1
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Function to add checkboxes
def add_checkbox(doc_id, text):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to add a checkbox
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'paragraphStyle': {'namedStyleType': 'NORMAL_TEXT'},
                'fields': 'namedStyleType'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'  # Functional checkbox
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Parse and add content with appropriate formatting
def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('# '):  # Heading 1
            add_text_to_doc(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_to_doc(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_to_doc(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_checkbox(doc_id, line[6:])  # Functional checkbox
        elif line.startswith('-'):  # Bullet point
            add_text_to_doc(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_to_doc(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_to_doc(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/1SckjOLZIpmhA166h4eNEUoUuB4KPvatls4wyvZ6t2Jw


In [ ]:
# v5 - user upload file

In [ ]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]  # Get the uploaded file name


Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960 (4).json


In [ ]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")

Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1eAnwczuD4g1ONMigmXpaJjptJk9CcJ_XWayL0rrkbnY


In [ ]:
# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)

Document shared with dani.s.ankit@gmail.com


In [ ]:
# Step 6: Upload and Read the Markdown File
print("Please upload your .md file containing the meeting notes.")
uploaded_md = files.upload()
md_file_name = list(uploaded_md.keys())[0]  # Get the uploaded .md file name

# Read the content of the uploaded file
with open(md_file_name, 'r') as file:
    meeting_notes = file.read()

print("Markdown file uploaded and read successfully!")


Please upload your .md file containing the meeting notes.


Saving meeting_notes.md to meeting_notes.md
Markdown file uploaded and read successfully!


In [ ]:
# Step 7: Add Content to Google Doc
def add_text_to_doc(doc_id, text, style='NORMAL_TEXT'):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to insert text at the end
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text) - 1
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Function to add checkboxes
def add_checkbox(doc_id, text):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to add a checkbox
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'paragraphStyle': {'namedStyleType': 'NORMAL_TEXT'},
                'fields': 'namedStyleType'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'  # Functional checkbox
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Parse and add content with appropriate formatting
def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('# '):  # Heading 1
            add_text_to_doc(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_to_doc(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_to_doc(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_checkbox(doc_id, line[6:])  # Functional checkbox
        elif line.startswith('-'):  # Bullet point
            add_text_to_doc(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_to_doc(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_to_doc(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/1eAnwczuD4g1ONMigmXpaJjptJk9CcJ_XWayL0rrkbnY


In [ ]:
# v6 - footer bold, @ italics

In [ ]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]  # Get the uploaded file name


Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960 (5).json


In [ ]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")

Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1TQXK3egoYdAFgbRoENSAay8Wj4kTq0iy_MIIrcd8Hc0


In [ ]:
# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)

Document shared with dani.s.ankit@gmail.com


In [ ]:
# Step 6: Upload and Read the Markdown File
print("Please upload your .md file containing the meeting notes.")
uploaded_md = files.upload()
md_file_name = list(uploaded_md.keys())[0]  # Get the uploaded .md file name

# Read the content of the uploaded file
with open(md_file_name, 'r') as file:
    meeting_notes = file.read()

print("Markdown file uploaded and read successfully!")


Please upload your .md file containing the meeting notes.


Saving meeting_notes.md to meeting_notes (1).md
Markdown file uploaded and read successfully!


In [ ]:
# Step 7: Add Content to Google Doc
def add_text_with_style(doc_id, text, style=None, bold=False, italic=False):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to insert text with styles
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'textStyle': {
                    'bold': bold,
                    'italic': italic
                },
                'fields': 'bold,italic'
            }
        }
    ]

    # Apply paragraph styles if provided
    if style:
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Function to add checkboxes
def add_checkbox(doc_id, text):
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Prepare requests to add a checkbox
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'  # Functional checkbox
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

# Parse and add content with appropriate formatting
def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    in_footer = False  # Track if we are in the footer section
    for line in lines:
        line = line.strip()
        if line.startswith('---'):  # Footer starts after ---
            in_footer = True
            continue

        if in_footer:  # Footer content
            add_text_with_style(doc_id, line, bold=True)
        elif line.startswith('# '):  # Heading 1
            add_text_with_style(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_with_style(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_with_style(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_checkbox(doc_id, line[6:])  # Functional checkbox
        elif '@' in line:  # Italicize @mentions
            parts = line.split('@')
            for i, part in enumerate(parts):
                if i == 0:
                    add_text_with_style(doc_id, part.strip(), italic=False)
                else:
                    name = part.split()[0]  # Extract @mention name
                    rest = part[len(name):].strip()
                    add_text_with_style(doc_id, '@' + name, italic=True)
                    if rest:
                        add_text_with_style(doc_id, rest, italic=False)
        elif line.startswith('-'):  # Bullet point
            add_text_with_style(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_with_style(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_with_style(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/1TQXK3egoYdAFgbRoENSAay8Wj4kTq0iy_MIIrcd8Hc0


In [ ]:
# v8 - footer part in footer section - not working

In [ ]:
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]


Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960.json


In [ ]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")


Authentication successful!


In [ ]:
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")

# Share the document
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

share_document_with_user(document_id, YOUR_EMAIL)


Document created! View it here: https://docs.google.com/document/d/1cQ6e2Jhx3pBTDw4wl-d6Wt5inFqAc2IdvLs2CE3xrhA
Document shared with dani.s.ankit@gmail.com


In [ ]:
print("Please upload your .md file containing the meeting notes.")
uploaded_md = files.upload()
md_file_name = list(uploaded_md.keys())[0]

with open(md_file_name, 'r') as file:
    meeting_notes = file.read()

print("Markdown file uploaded and read successfully!")


Please upload your .md file containing the meeting notes.


Saving meeting_notes.md to meeting_notes.md
Markdown file uploaded and read successfully!


In [ ]:
# Step 6: Add Footer Section
def add_footer(doc_id, footer_text):
    # Insert footer content at the end of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': footer_text + '\n'
            }
        },
        {
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index - 1 + len(footer_text)
                },
                'textStyle': {'bold': True},
                'fields': 'bold'
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()


In [ ]:
# Step 7: Add Content to the Document
def add_text_with_style(doc_id, text, style=None, bold=False, italic=False):
    if not text.strip():  # Skip empty or whitespace-only text
        return

    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        }
    ]

    if bold or italic:
        requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index - 1 + len(text)
                },
                'textStyle': {
                    'bold': bold,
                    'italic': italic
                },
                'fields': 'bold,italic'
            }
        })

    if style:
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index - 1 + len(text)
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def parse_markdown_and_add_content(doc_id, markdown):
    lines = markdown.split('\n')
    in_footer = False
    footer_text = ""

    for line in lines:
        line = line.strip()

        if line.startswith('---'):  # Footer starts after ---
            in_footer = True
            continue

        if in_footer:  # Collect footer content
            footer_text += line + '\n'
        elif line.startswith('# '):  # Heading 1
            add_text_with_style(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_with_style(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_with_style(doc_id, line[4:], style='HEADING_3')
        elif '@' in line:  # Italicize @mentions
            parts = line.split('@')
            for i, part in enumerate(parts):
                if i == 0 and part.strip():
                    add_text_with_style(doc_id, part)
                elif part.strip():
                    mention, rest = part.split(' ', 1) if ' ' in part else (part, '')
                    add_text_with_style(doc_id, f"@{mention}", italic=True)
                    if rest.strip():
                        add_text_with_style(doc_id, rest)
        elif line.startswith('-'):  # Bullet point
            add_text_with_style(doc_id, f"• {line[1:].strip()}")
        elif line.startswith('*'):  # Nested bullet
            add_text_with_style(doc_id, f"    • {line[1:].strip()}")
        elif line:  # Regular text
            add_text_with_style(doc_id, line)

    # Add the footer content
    if footer_text.strip():
        add_footer(doc_id, footer_text.strip())

In [ ]:
# Step 8: Add Content to the Document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


Content added to document! View it here: https://docs.google.com/document/d/1cQ6e2Jhx3pBTDw4wl-d6Wt5inFqAc2IdvLs2CE3xrhA


In [ ]:
# v9 - @ underline

In [ ]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [ ]:
# Step 2: Upload JSON Key
from google.colab import files

# Upload the service account JSON key file
uploaded = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded.keys())[0]  # Get the uploaded file name


Saving river-tiger-448802-b8-c2bb3940d960.json to river-tiger-448802-b8-c2bb3940d960 (3).json


In [ ]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define scopes for Google Docs and Drive
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the uploaded service account JSON file
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")


Authentication successful!


In [ ]:
# Step 4: Create a New Google Doc
# Create the document
document = docs_service.documents().create(body={'title': 'Product Team Sync'}).execute()
document_id = document.get('documentId')
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1NuBRGjfIIu_nJeulSZOhYE9CZ-4dJs2nitXztDLpPVA


In [ ]:
# Step 5: Share the Document with Your Email
YOUR_EMAIL = 'dani.s.ankit@gmail.com'  # Replace with your email address

# Share the document with your email
def share_document_with_user(doc_id, email):
    permissions = {
        'type': 'user',
        'role': 'writer',  # Options: 'writer' or 'reader'
        'emailAddress': email
    }
    drive_service.permissions().create(
        fileId=doc_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {email}")

# Call the function to share the document
share_document_with_user(document_id, YOUR_EMAIL)

Document shared with dani.s.ankit@gmail.com


In [ ]:
# Step 6: Upload and Read the Markdown File
print("Please upload your .md file containing the meeting notes.")
uploaded_md = files.upload()
md_file_name = list(uploaded_md.keys())[0]  # Get the uploaded .md file name

# Read the content of the uploaded file
with open(md_file_name, 'r') as file:
    meeting_notes = file.read()

print("Markdown file uploaded and read successfully!")


Please upload your .md file containing the meeting notes.


Saving meeting_notes.md to meeting_notes (4).md
Markdown file uploaded and read successfully!


In [ ]:
def add_text_with_style(doc_id, text, style=None, bold=False, underline=False, color=None):
    """
    Adds text to the Google Doc with optional paragraph and text styles.
    """
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Insert text
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        }
    ]

    # Apply paragraph styles if provided
    if style:
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        })

    # Apply text styles (bold, underline, color)
    text_style = {}
    if bold:
        text_style['bold'] = True
    if underline:
        text_style['underline'] = True
    if color:
        text_style['foregroundColor'] = {
            'color': {
                'rgbColor': {
                    'red': color[0],
                    'green': color[1],
                    'blue': color[2]
                }
            }
        }

    if text_style:
        requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'textStyle': text_style,
                'fields': ','.join(text_style.keys())
            }
        })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def add_checkbox(doc_id, text):
    """
    Adds a functional checkbox to the Google Doc.
    """
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'  # Functional checkbox
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def parse_markdown_and_add_content(doc_id, markdown):
    """
    Parses markdown and adds styled content to the Google Doc.
    """
    lines = markdown.split('\n')
    in_footer = False  # Track if we are in the footer section
    for line in lines:
        line = line.strip()
        if line.startswith('---'):  # Footer starts after ---
            in_footer = True
            continue

        if in_footer:  # Footer content
            add_text_with_style(doc_id, line, bold=True)
        elif line.startswith('# '):  # Heading 1
            add_text_with_style(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_with_style(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_with_style(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_checkbox(doc_id, line[6:])  # Functional checkbox
        elif '@' in line:
            # Improved @ mention handling
            parts = []
            start = 0
            while '@' in line[start:]:
                at_index = line.find('@', start)

                # Add text before @ with default styling
                if at_index > start:
                    parts.append((line[start:at_index], False))

                # Find end of mention (space or end of line)
                space_index = line.find(' ', at_index)
                if space_index == -1:
                    space_index = len(line)

                # Add mention with special styling
                parts.append((line[at_index:space_index], True))

                # Update start for next iteration
                start = space_index

            # Add any remaining text
            if start < len(line):
                parts.append((line[start:], False))

            # Add parts with appropriate styling
            for text, is_mention in parts:
                if is_mention:
                    add_text_with_style(
                        doc_id,
                        text,
                        underline=False,
                        color=(0.0, 0.0, 1.0)  # Blue color for @mentions
                    )
                else:
                    add_text_with_style(doc_id, text)
        elif line.startswith('-'):  # Bullet point
            add_text_with_style(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_with_style(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_with_style(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")

Content added to document! View it here: https://docs.google.com/document/d/1NuBRGjfIIu_nJeulSZOhYE9CZ-4dJs2nitXztDLpPVA


In [ ]:
def add_text_with_style(doc_id, text, style=None, bold=False, underline=False, color=None):
    """
    Adds text to the Google Doc with optional paragraph and text styles.
    """
    # Get the current end index of the document
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Insert text
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        }
    ]

    # Apply paragraph styles if provided
    if style:
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'paragraphStyle': {'namedStyleType': style},
                'fields': 'namedStyleType'
            }
        })

    # Apply text styles (bold, underline, color)
    text_style = {}
    if bold:
        text_style['bold'] = True
    if underline:
        text_style['underline'] = True
    if color:
        text_style['foregroundColor'] = {
            'color': {
                'rgbColor': {
                    'red': color[0],
                    'green': color[1],
                    'blue': color[2]
                }
            }
        }

    if text_style:
        requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'textStyle': text_style,
                'fields': ','.join(text_style.keys())
            }
        })

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def add_checkbox(doc_id, text):
    """
    Adds a functional checkbox to the Google Doc.
    """
    doc = docs_service.documents().get(documentId=doc_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},
                'text': text + '\n'
            }
        },
        {
            'createParagraphBullets': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'bulletPreset': 'BULLET_CHECKBOX'  # Functional checkbox
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

def parse_markdown_and_add_content(doc_id, markdown):
    """
    Parses markdown and adds styled content to the Google Doc.
    """
    lines = markdown.split('\n')
    in_footer = False  # Track if we are in the footer section
    for line in lines:
        line = line.strip()
        if line.startswith('---'):  # Footer starts after ---
            in_footer = True
            continue

        if in_footer:  # Footer content
            add_text_with_style(doc_id, line, bold=True)
        elif line.startswith('# '):  # Heading 1
            add_text_with_style(doc_id, line[2:], style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_with_style(doc_id, line[3:], style='HEADING_2')
        elif line.startswith('### '):  # Heading 3
            add_text_with_style(doc_id, line[4:], style='HEADING_3')
        elif line.startswith('- [ ]'):  # Checkbox
            add_checkbox(doc_id, line[6:])  # Functional checkbox
        elif '@' in line:
            # Completely new approach to @ mentions
            current_pos = 0
            while current_pos < len(line):
                # Find next @ mention
                at_index = line.find('@', current_pos)

                if at_index == -1:
                    # No more @ mentions, add remaining text normally
                    add_text_with_style(doc_id, line[current_pos:])
                    break

                # Add text before @ normally
                if at_index > current_pos:
                    add_text_with_style(doc_id, line[current_pos:at_index])

                # Find end of mention (space or end of line)
                space_index = line.find(' ', at_index)
                if space_index == -1:
                    space_index = len(line)

                # Add mention with special styling
                mention = line[at_index:space_index]
                add_text_with_style(
                    doc_id,
                    mention,
                    underline=True,
                    color=(0.0, 0.0, 1.0)  # Blue color
                )

                # Update position
                current_pos = space_index
        elif line.startswith('-'):  # Bullet point
            add_text_with_style(doc_id, f"• {line[1:].strip()}", style='NORMAL_TEXT')
        elif line.startswith('*'):  # Nested bullet
            add_text_with_style(doc_id, f"    • {line[1:].strip()}", style='NORMAL_TEXT')
        elif line:  # Regular text
            add_text_with_style(doc_id, line, style='NORMAL_TEXT')

# Add content to the document
parse_markdown_and_add_content(document_id, meeting_notes)

print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")

Content added to document! View it here: https://docs.google.com/document/d/1NuBRGjfIIu_nJeulSZOhYE9CZ-4dJs2nitXztDLpPVA


In [ ]:
# code woith comemtns

In [1]:
# Step 1: Install Dependencies
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [2]:
# Step 2: Upload Service Account JSON Key
from google.colab import files

# Prompt the user to upload the JSON key file
print("Upload the Google Cloud service account JSON key file:")
uploaded_key = files.upload()
SERVICE_ACCOUNT_FILE = list(uploaded_key.keys())[0]  # Extract uploaded file name


Upload the Google Cloud service account JSON key file:


Saving resonant-rock-448902-j3-a8e32e697012.json to resonant-rock-448902-j3-a8e32e697012.json


In [3]:
# Step 3: Authenticate with Google Docs API
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# Define the required scopes for Google Docs and Google Drive APIs
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Authenticate using the service account JSON key
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Initialize the Google Docs and Drive services
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

print("Authentication successful!")

Authentication successful!


In [4]:
# Step 4: Create a New Google Doc
# Create a new Google Doc with a specified title
document_metadata = docs_service.documents().create(body={'title': 'Meeting Notes Document'}).execute()
document_id = document_metadata.get('documentId')  # Get the document ID
print(f"Document created! View it here: https://docs.google.com/document/d/{document_id}")


Document created! View it here: https://docs.google.com/document/d/1jvRyZAwKnP_IV8psomQG1zO0Id3W0HAOXjVFWkgtomo


In [5]:
# Step 5: Share the Document with the User
USER_EMAIL = 'dani.s.ankit@gmail.com'  # Replace this with your email address

def share_document(document_id, user_email):
    """
    Share the Google Document with the specified email address.
    """
    permissions = {
        'type': 'user',
        'role': 'writer',  # Grant 'writer' access to the user
        'emailAddress': user_email
    }
    drive_service.permissions().create(
        fileId=document_id,
        body=permissions,
        fields='id'
    ).execute()
    print(f"Document shared with {user_email}")

# Share the document with the specified email
share_document(document_id, USER_EMAIL)

Document shared with dani.s.ankit@gmail.com


In [6]:
# Step 6: Upload the Markdown File
print("Please upload your .md file containing the meeting notes:")
uploaded_markdown = files.upload()
markdown_file_name = list(uploaded_markdown.keys())[0]  # Extract uploaded file name

# Read the content of the uploaded Markdown file
with open(markdown_file_name, 'r') as file:
    meeting_notes_content = file.read()

print("Markdown file uploaded and read successfully!")

Please upload your .md file containing the meeting notes:


Saving meeting_notes.md to meeting_notes.md
Markdown file uploaded and read successfully!


In [7]:
# Step 7: Helper Function to Add Styled Text
def add_text_with_style_batch(document_id, text, paragraph_style=None, bold=False, underline=False, font_color=None):
    """
    Adds styled text to the Google Doc and ensures the range is valid.
    """
    # Fetch the document's current end index
    doc = docs_service.documents().get(documentId=document_id).execute()
    end_index = doc.get('body').get('content')[-1].get('endIndex')

    # Insert the text first
    requests = [
        {
            'insertText': {
                'location': {'index': end_index - 1},  # Insert at the end
                'text': text + '\n'
            }
        }
    ]

    # Apply paragraph styles if specified
    if paragraph_style:
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)  # Ensure range covers the inserted text
                },
                'paragraphStyle': {'namedStyleType': paragraph_style},
                'fields': 'namedStyleType'
            }
        })

    # Apply text styles (bold, underline, font color)
    text_style = {}
    if bold:
        text_style['bold'] = True
    if underline:
        text_style['underline'] = True
    if font_color:
        text_style['foregroundColor'] = {
            'color': {'rgbColor': {'red': font_color[0], 'green': font_color[1], 'blue': font_color[2]}}
        }

    # Ensure valid range and apply styles
    if text_style:
        requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': end_index - 1,
                    'endIndex': end_index + len(text)
                },
                'textStyle': text_style,
                'fields': ','.join(text_style.keys())
            }
        })

    # Execute all requests as a batch
    docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

# Step 8: Parse Markdown and Add Content to Google Doc
def parse_markdown_and_add_content(document_id, markdown_content):
    """
    Parses the markdown content and adds styled content to the Google Doc.
    """
    lines = markdown_content.split('\n')
    is_footer = False  # Tracks if we are in the footer section

    for line in lines:
        line = line.strip()
        if line.startswith('---'):  # Detect footer section
            is_footer = True
            continue

        if is_footer:  # Footer content is bold
            add_text_with_style_batch(document_id, line, bold=True)
        elif line.startswith('# '):  # Heading 1
            add_text_with_style_batch(document_id, line[2:], paragraph_style='HEADING_1')
        elif line.startswith('## '):  # Heading 2
            add_text_with_style_batch(document_id, line[3:], paragraph_style='HEADING_2')
        elif line.startswith('- [ ]'):  # Checkbox
            add_text_with_style_batch(document_id, line[6:])
        elif '@' in line:  # Style @mentions
            words = line.split(' ')
            for word in words:
                if word.startswith('@'):  # Apply style to mentions
                    add_text_with_style_batch(document_id, word, underline=True, font_color=(0.0, 0.0, 1.0))
                else:
                    add_text_with_style_batch(document_id, word)
        else:  # Regular text
            add_text_with_style_batch(document_id, line)

# Add parsed markdown content to the document
parse_markdown_and_add_content(document_id, meeting_notes_content)
print(f"Content added to document! View it here: https://docs.google.com/document/d/{document_id}")


HttpError: <HttpError 429 when requesting https://docs.googleapis.com/v1/documents/1jvRyZAwKnP_IV8psomQG1zO0Id3W0HAOXjVFWkgtomo:batchUpdate?alt=json returned "Quota exceeded for quota metric 'Quota group for write operations' and limit 'Quota group for write operations per minute per user' of service 'docs.googleapis.com' for consumer 'project_number:495496367459'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_limit_value': '60', 'service': 'docs.googleapis.com', 'consumer': 'projects/495496367459', 'quota_metric': 'docs.googleapis.com/write_requests', 'quota_limit': 'WriteRequestsPerMinutePerUser', 'quota_location': 'global'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">